### GetResult.py file finally

Function1:Determine CF or CB

Function2:get CB result(Embedding+BERD)

Function3:load CF model & return result

In [1]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl


In [2]:
from google.cloud import bigquery
from google.cloud import storage
import pickle
import pandas as pd
import surprise

In [3]:
def query_data(sql_query,user_id):
    client = bigquery.Client(project='brave-watch-414204')
    
    # Define query parameters
    query_params = [
        bigquery.ScalarQueryParameter("user_id", "INT64", int(user_id))
    ]

    # Execute the query job
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    query_job = client.query(sql_query, job_config=job_config)

    # Fetch the result
    result = query_job.result()

    return result

In [4]:
# Function 1
# Return 'CF' or 'CB' based on how many recipes did this user rate before
# input parm: user_id -> which will be send by the end application
def determine_rcs_model(user_id):
    # Initialize BigQuery client
    client = bigquery.Client(project='brave-watch-414204')

    sql_query = """
    SELECT COUNT(DISTINCT recipe_id) AS num_occurrences
    FROM `brave-watch-414204.RecipeQuery.interactions`
    WHERE user_id = @user_id
    """
    
    # Define query parameters
    query_params = [
        bigquery.ScalarQueryParameter("user_id", "INT64", int(user_id))
    ]

    # Execute the query job
    job_config = bigquery.QueryJobConfig(query_parameters=query_params)
    query_job = client.query(sql_query, job_config=job_config)

    # Fetch the result
    result = query_job.result()
    num_occurrences = list(result)[0].num_occurrences

    if num_occurrences > 115:
        return 'CF'
    else:
        return 'CB'

In [5]:
# Function 2
# this part will be done by jh & th
def get_CB_model():
    return model

In [6]:
# Function 3
# load the trained CF model from GCS Bucket -> the model is saved by the pipeline part
def get_CF_model(bucket_name,blob_name):
    
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    
    blob = bucket.blob(blob_name)
    
    pickle_file = blob.download_as_bytes()

    model = pickle.loads(pickle_file)
    
    return model

In [7]:
def process_CF_result(model,cust_user_id):
    # get recipes list
    sql_query_rated_recipe = """
        SELECT distinct recipe_id
        FROM `brave-watch-414204.RecipeQuery.interactions`
        WHERE user_id = @user_id
    """

    sql_query_all_recipe = """
        SELECT distinct recipe_id
        FROM `brave-watch-414204.RecipeQuery.interactions`
    """
    rated_recipes_id_result = query_data(sql_query_rated_recipe,cust_user_id)
    all_recipes_id_result = query_data(sql_query_all_recipe,cust_user_id)

    rated_recipes_id = []
    for row in rated_recipes_id_result:
        rated_recipes_id.append(row[0])

    all_recipes_id = []
    for row in all_recipes_id_result:
        all_recipes_id.append(row[0])

    unrated_recipes_list = list(set(all_recipes_id) - set(rated_recipes_id))

    predictions = []
    for recipe_id in unrated_recipes_list:
        prediction = model.predict(cust_user_id, recipe_id)
        predictions.append({'recipe_id': recipe_id, 'predicted_rating': prediction.est})

    predictions_df = pd.DataFrame(predictions)
    # Sort recipes by predicted ratings in descending order
    predictions_df = predictions_df.sort_values(by='predicted_rating', ascending=False)

    # get the recipes name and description
    sql_query_recipe_details = """
    SELECT  
      distinct id,
      name,
      description
    FROM `recipe-recommendation-2024.RecipeQuery.recipes` 
    """

    recipes_detail_result = query_data(sql_query_recipe_details,'0')
    recipes_detail = recipes_detail_result.to_dataframe()

    final_predictions_df = pd.merge(predictions_df,recipes_detail,left_on='recipe_id',right_on='id',how='left')

    top5_rec = []
    for value in final_predictions_df.head().values:
        top5_rec.append({'id':value[0],'description':value[4],'name':value[3]})

    final_result = {"recipes":top5_rec}
    return final_result

In [8]:
#This part will be done by jh&th
def process_CB_results(model,cust_user_id):

    #Get all recipe_id rates by user
    sql_query_recipe_rated="""
        SELECT distinct recipe_id
        FROM `recipe-recommendation-2024.RecipeQuery.interactions`
        WHERE user_id = @user_id
"""

    #Embed user's all review
    sql_query_embed_rated_recipe_review = """
with tmp as (
    SELECT user_id,
           STRING_AGG(review, ' ') AS aggregated_reviews
    FROM `brave-watch-414204.RecipeQuery.interactions` 
    WHERE user_id = @user_id
)

SELECT *
FROM
ML.GENERATE_EMBEDDING(
  MODEL `RecipeQuery.gecko_model`,
  (SELECT aggregated_reviews as content, user_id as title FROM tmp),
  STRUCT(TRUE AS flatten_json_output)
)
"""
    #Get all recipe_id 
    sql_query_recipe_all="""
        SELECT distinct recipe_id
        FROM `recipe-recommendation-2024.RecipeQuery.interactions`
"""
    #embed all recipes' description
    sql_query_embed_all_recipe_description="""
    
""" 

    sql_query_embed_rated_recipe_review = query_data(sql_query_embed_rated_recipe_review,cust_user_id)
    sql_query_embed_all_recipe_description = query_data(sql_query_embed_all_recipe_description,cust_user_id)
    
    #list all the recipe_id user rated
    rated_recipes_id = []
    for row in sql_query_recipe_rated:
        rated_recipes_id.append(row[0])

    #list all the recipe_id
    all_recipes_id = []
    for row in sql_query_recipe_all:
        all_recipes_id.append(row[0])
    
    #all recipes reduce those rated by users are the source we can use to predict
    unrated_recipes_list = list(set(all_recipes_id) - set(rated_recipes_id))

    predictions = []
    for recipe_id in unrated_recipes_list:
        prediction = model.predict(cust_user_id, recipe_id)
        predictions.append({'recipe_id': recipe_id, 'predicted_rating': prediction.est})
    
    predictions_df = pd.DataFrame(predictions)
    # Sort recipes by predicted ratings in descending order
    predictions_df = predictions_df.sort_values(by='predicted_rating', ascending=False)

    # get the recipes name and description
    sql_query_recipe_details = """
    SELECT  
      distinct id,
      name,
      description
    FROM `recipe-recommendation-2024.RecipeQuery.recipes`
    where id = 
    """

    recipes_detail_result = query_data(sql_query_recipe_details,'0')
    recipes_detail = recipes_detail_result.to_dataframe()

    final_predictions_df = pd.merge(predictions_df,recipes_detail,left_on='recipe_id',right_on='id',how='left')

    top5_rec = []
    for value in final_predictions_df.head().values:
        top5_rec.append({'id':value[0],'description':value[4],'name':value[3]})

    final_result = {"recipes":top5_rec}
    return final_result



In [9]:
# Integrated Function
# return the Final result
def main(cust_user_id):
    flag = determine_rcs_model(cust_user_id)
    if flag == 'CB':
        model = get_CB_model()
    elif flag == 'CF':
        # get model
        model = get_CF_model('brave-watch-414204','Model/knn_model.pkl')
        final_result = process_CF_result(model,cust_user_id)
        
        return final_result
    else:
        raise ValueError(f"Unknown flag value: {flag}")
    return None

In [12]:
CF = main('424680')
CF

NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/brave-watch-414204/o/Model%2Fknn_model.pkl?alt=media: No such object: brave-watch-414204/Model/knn_model.pkl: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)